# Human review
Combine AI result into excel for human review

In [ ]:
%pip install pandas openpyxl xlsxwriter

In [ ]:
base_folder = "data/ITE3101_practical_tests/ite-3101-practical-test-ab-submissions/"

In [ ]:
import glob
import pandas as pd

xlsx_files = glob.glob(base_folder + "student_score_*.xlsx")
file_names = [file.split("/")[-1] for file in xlsx_files]
print(file_names)
file_dict = {file_name.replace("student_score_", "").replace(
    ".xlsx", ""): file_name for file_name in file_names}
print(file_dict)
dataframes = {}
for key, value in file_dict.items():
    dataframes[key] = pd.read_excel(base_folder+value)

dataframes

In [ ]:
first_dataframe = next(iter(dataframes.values()))
questions = [col.replace(" Content", "")
             for col in first_dataframe.columns if col.endswith(" Content")]
questions

In [ ]:
base_dataframe = first_dataframe[['Directory',
                                  'Commit Time', 'After Deadline']].copy()
base_dataframe.head()

In [ ]:
new_dataframes = dataframes.copy()

question_scores = {}
for model, report in new_dataframes.items():
    for question in questions:
        # get the column name containing the question
        question_cols = ['Directory'] + \
            [col for col in report.columns if question in col]
        question_df = report[question_cols]
        # remove the question from the column name and trim the column name
        question_df.columns = [col.replace(
            question, "").strip() for col in question_df.columns]
        # excpet for Directory, Content and Commit ,add model as prefix to the column name
        question_df.columns = [model + " " + col if col not in ['Directory',
                                                                'Content', 'Commit'] else col for col in question_df.columns]

        # print(question, question_df.columns)
        if question not in question_scores:
            question_scores[question] = question_df
        else:
            # for duplicated column names, only keep the first one
            question_scores[question] = pd.merge(question_scores[question], question_df, on=[
                                                 'Directory', 'Content', 'Commit'], how='outer')

In [ ]:
for question, dataframe in question_scores.items():
    # Move "Content" column to last column
    dataframe = dataframe[[
        col for col in dataframe.columns if col != "Content"] + ["Content"]]
    # In Content column, replace carriage returns to line feed
    dataframe["Content"] = dataframe["Content"].str.replace("\n", "\r\n")

    # Calculate the average of columns ending with "Score"
    score_columns = [col for col in dataframe.columns if col.endswith("Score")]
    dataframe["Average"] = dataframe[score_columns].mean(axis=1)
    dataframe["Score"] = dataframe[score_columns].mean(axis=1)

    # Update the dataframe in question_scores
    question_scores[question] = dataframe

In [ ]:
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows

# Create a new Excel file
writer = pd.ExcelWriter(
    base_folder+'human_score_review.xlsx', engine='xlsxwriter')


# Write each question_scores dataframe to a separate sheet
for question, dataframe in question_scores.items():
    dataframe.to_excel(writer, sheet_name=question, index=False)
    # get the worksheet for the current question
    worksheet = writer.sheets[question]
    row_heights = (dataframe["Content"].str.count("\r\n") + 1) * 15
    # set height of each row to 12.75 points times the number of lines in the content
    for row, height in enumerate(row_heights):
        worksheet.set_row(row+1, height)
     # set column width of "Content" column to the longest line split by line feed
    max_content_width = dataframe["Content"].str.split(
        "\r\n").str.len().max() * 2
    # get the index of the "Content" column
    content_col = dataframe.columns.get_loc("Content")
    # print(question, max_content_width, content_col)
    worksheet.set_column(content_col, content_col, max_content_width)

    for model in new_dataframes.keys():
        worksheet.set_column(
            dataframe.columns.get_loc(model + " Calculation"), dataframe.columns.get_loc(model + " Calculation"), 60)
        worksheet.set_column(
            dataframe.columns.get_loc(model + " Comments"), dataframe.columns.get_loc(model + " Comments"), 60)


# Save the Excel file
writer.close()